In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE  # SMOTE for oversampling minority class
from imblearn.ensemble import EasyEnsembleClassifier  # EasyEnsemble for ensemble learning on imbalanced datasets
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autoencoder-features/top_0.25_percent_features_cv.csv
/kaggle/input/autoencoder-features/top_0.5_percent_features_cv.csv
/kaggle/input/autoencoder-features/top_1.0_percent_features_cv.csv
/kaggle/input/pancancer-8-dataset/genes.csv
/kaggle/input/pancancer-8-dataset/8_cancers_merged.csv
/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv


In [ ]:
df = pd.read_csv('/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv')
df = df.iloc[:, 1:]
temp = df.iloc[:, :-1]
null_rows = temp[temp.isna().all(axis=1)].index
print(null_rows)
df.drop(index=[1226, 1741, 1917, 2069, 2494, 3084, 3637], inplace=True)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

Index([1226, 1741, 1917, 2069, 2494, 3084, 3637], dtype='int64')


In [ ]:
genes = ['GNB1',
'TAS1R1',
'CDK11A',
'NOC2L' ,
'CEP104' ,
'GPR153' ,
'DFFB' ,
'TPRG1L',
'RER1',
'LRRC47',
'AURKAIP1',
'MRPL20' ,
'ZBTB48']


In [ ]:
X = X[genes]
X.shape

(4104, 13)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

In [ ]:
# Apply SMOTE to balance the class distribution
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_train_resampled.shape

(7848, 13)

In [ ]:
# Compute class weights for the training data
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = dict(zip(np.unique(y_train_resampled), class_weights))

# Example: SVM with class weights
svm_model = SVC(class_weight=class_weight_dict, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier

# Models setup
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')  # Using class_weight='balanced' to address imbalance
svm_model = SVC(class_weight='balanced', random_state=42,probability=True)
#xgb_model = XGBClassifier(scale_pos_weight=10, random_state=42)  # Adjust scale_pos_weight if needed
ada_model = AdaBoostClassifier(random_state=42)
lr_model = LogisticRegression(random_state=42)
et_model = ExtraTreesClassifier(random_state=42,n_estimators=100)
dt_model = DecisionTreeClassifier(random_state=42)



# Training models
rf_model.fit(X_train_resampled, y_train_resampled)
svm_model.fit(X_train_resampled, y_train_resampled)
#xgb_model.fit(X_train_resampled, y_train_resampled)
ada_model.fit(X_train_resampled, y_train_resampled)
lr_model.fit(X_train_resampled, y_train_resampled)
dt_model.fit(X_train_resampled, y_train_resampled)
et_model.fit(X_train_resampled, y_train_resampled)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ExtraTreesClassifier(random_state=42)

In [ ]:
from sklearn.ensemble import VotingClassifier
eclfs = VotingClassifier(estimators=[('RF', rf_model), ('ET', et_model), ('lr', lr_model)],voting='soft')

In [ ]:
estimators = [('RF', rf_model),('ET',et_model ),('LR',lr_model)]
stackclassifier = StackingClassifier(estimators=estimators, final_estimator=svm_model)

In [ ]:
eclfs.fit(X_train_resampled, y_train_resampled)
stackclassifier.fit(X_train_resampled, y_train_resampled)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

StackingClassifier(estimators=[('RF',
                                RandomForestClassifier(class_weight='balanced',
                                                       random_state=42)),
                               ('ET', ExtraTreesClassifier(random_state=42)),
                               ('LR', LogisticRegression(random_state=42))],
                   final_estimator=SVC(class_weight='balanced',
                                       probability=True, random_state=42))

In [ ]:
# Make predictions
rf_preds = rf_model.predict(X_test)
svm_preds = svm_model.predict(X_test)
#xgb_preds = xgb_model.predict(X_test)
#ada_preds = ada_model.predict(X_test)
lr_preds = lr_model.predict(X_test)
et_preds = et_model.predict(X_test)
st_preds = stackclassifier.predict(X_test)
soft_voting = eclfs.predict(X_test)

# Evaluate all models
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Evaluation:")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision (Macro Average):", precision_score(y_true, y_pred, average='macro'))
    print("Recall (Macro Average):", recall_score(y_true, y_pred, average='macro'))
    print("F1 Score (Macro Average):", f1_score(y_true, y_pred, average='macro'))
    print("\n")

# Evaluate each model
evaluate_model(y_test, rf_preds, 'Random Forest')
evaluate_model(y_test, svm_preds, 'SVM')
#evaluate_model(y_test, xgb_preds, 'XGBoost')
#evaluate_model(y_test, ada_preds, 'AdaBoost')
evaluate_model(y_test, lr_preds, 'Logistic Regression')
evaluate_model(y_test, et_preds, 'Extra tree classifier')
evaluate_model(y_test, st_preds, 'Stacking Classifier')
evaluate_model(y_test, soft_voting, "Soft Voting")

Random Forest Evaluation:
Accuracy: 0.8574908647990256
Precision (Macro Average): 0.8814526011797283
Recall (Macro Average): 0.8896575754599036
F1 Score (Macro Average): 0.8852307589328021


SVM Evaluation:
Accuracy: 0.8696711327649208
Precision (Macro Average): 0.8988946015434143
Recall (Macro Average): 0.9018656550994242
F1 Score (Macro Average): 0.9000970883686913


Logistic Regression Evaluation:
Accuracy: 0.8343483556638246
Precision (Macro Average): 0.8669811811704116
Recall (Macro Average): 0.8830388039706231
F1 Score (Macro Average): 0.8722349269655619


Extra tree classifier Evaluation:
Accuracy: 0.8684531059683313
Precision (Macro Average): 0.8964868017889263
Recall (Macro Average): 0.8977285951909513
F1 Score (Macro Average): 0.8968681397118444


Stacking Classifier Evaluation:
Accuracy: 0.8684531059683313
Precision (Macro Average): 0.9071809772296016
Recall (Macro Average): 0.8867650572491484
F1 Score (Macro Average): 0.8949824433832709


Soft Voting Evaluation:
Accuracy: 0

In [ ]:
from sklearn.metrics import matthews_corrcoef

# Evaluate all models with MCC
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Evaluation:")
    #print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision (Macro Average):", precision_score(y_true, y_pred, average='macro'))
    print("Recall (Macro Average):", recall_score(y_true, y_pred, average='macro'))
    print("F1 Score (Macro Average):", f1_score(y_true, y_pred, average='macro'))
    print("Matthews Correlation Coefficient (MCC):", matthews_corrcoef(y_true, y_pred))
    print("\n")

# Evaluate each model
evaluate_model(y_test, rf_preds, 'Random Forest')
evaluate_model(y_test, svm_preds, 'SVM')
#evaluate_model(y_test, xgb_preds, 'XGBoost')
#evaluate_model(y_test, ada_preds, 'AdaBoost')
evaluate_model(y_test, lr_preds, 'Logistic Regression')
evaluate_model(y_test, et_preds, 'Extra tree classifier')
evaluate_model(y_test, st_preds, 'Stacking Classifier')
evaluate_model(y_test, soft_voting, "Soft Voting")


Random Forest Evaluation:
Precision (Macro Average): 0.8814526011797283
Recall (Macro Average): 0.8896575754599036
F1 Score (Macro Average): 0.8852307589328021
Matthews Correlation Coefficient (MCC): 0.8284019070729548


SVM Evaluation:
Precision (Macro Average): 0.8988946015434143
Recall (Macro Average): 0.9018656550994242
F1 Score (Macro Average): 0.9000970883686913
Matthews Correlation Coefficient (MCC): 0.843466817331964


Logistic Regression Evaluation:
Precision (Macro Average): 0.8669811811704116
Recall (Macro Average): 0.8830388039706231
F1 Score (Macro Average): 0.8722349269655619
Matthews Correlation Coefficient (MCC): 0.8048180530669493


Extra tree classifier Evaluation:
Precision (Macro Average): 0.8964868017889263
Recall (Macro Average): 0.8977285951909513
F1 Score (Macro Average): 0.8968681397118444
Matthews Correlation Coefficient (MCC): 0.8412924603533897


Stacking Classifier Evaluation:
Precision (Macro Average): 0.9071809772296016
Recall (Macro Average): 0.886765057